# CNN-LSTM Model for Solar Irradiance Forecasting (15-min Resolution)

This notebook supports the results presented in the related journal article.

---

## 1. Libraries and Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import datetime
import itertools
import h5py
import matplotlib.dates as mdates

from keras.utils import plot_model


%matplotlib inline

In [ ]:
print("tensorflow version:", tf.__version__)
gpus =  tf.config.list_physical_devices('GPU')
print("available gpus:", gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print("available cpus:", os.cpu_count())

In [ ]:
cwd = "F:/SIFS" 

data_folder = os.path.join(cwd,"data_expanded_15min")
data_path = os.path.join(data_folder, "forecast_dataset.hdf5")

model_name = 'SIFS_forecast_l5min'
output_folder = os.path.join(cwd,"model_output_15min", model_name)
if os.path.isdir(output_folder)==False:
    os.makedirs(output_folder)


print("data_folder:", data_folder)
print("data_path:", data_path)
print("output_folder:", output_folder)

In [ ]:
forecast_dataset = h5py.File(data_path, 'r')

def get_all(name):
    if name!=None:
        print(forecast_dataset[name])
    
forecast_dataset.visit(get_all)

In [ ]:
print('-'*50)
img_side_len = forecast_dataset['trainval']['images_log'].shape[2]
num_log_term = forecast_dataset['trainval']['images_log'].shape[1]
num_color_channel =forecast_dataset['trainval']['images_log'].shape[4]
image_input_dim = [img_side_len,img_side_len,num_log_term*num_color_channel]

print("image side length:", img_side_len)
print("number of log terms:", num_log_term)
print("number of color channels:", num_color_channel)
print("input image dimension:", image_input_dim)

times_trainval = np.load(os.path.join(data_folder,"forecast_data","times_trainval.npy"),allow_pickle=True)
print("times_trainval.shape:", times_trainval.shape)

In [ ]:
num_samples = len(times_trainval)
batch_size = num_samples//5
indices = np.arange(num_samples)
print('-'*50)
print('data reading start...')
for i in range(num_samples // batch_size):
    start_time = time.time()
    start_idx = (i * batch_size) % num_samples
    if i<num_samples // batch_size-1:
        idxs = indices[start_idx:start_idx + batch_size]
    else:
        idxs = indices[start_idx:]
    _ = forecast_dataset['trainval']['images_log'][idxs]
    _ = forecast_dataset['trainval']['si_pred'][idxs]
    _ = forecast_dataset['trainval']['cloud_fraction'][idxs]
    end_time = time.time()
    print("batch {0} samples: {1} to {2}, {3:.2f}% finished, processing time {4:.2f}s"
          .format(i+1, idxs[0],idxs[-1],(idxs[-1]/num_samples)*100,(end_time-start_time)))

### Input data pipeline helper functions

In [ ]:
def day_block_shuffle(times_trainval):
    
    dates_trainval = np.zeros_like(times_trainval, dtype=datetime.date)
    for i in range(len(times_trainval)):
        dates_trainval[i] = times_trainval[i].date()

    unique_dates = np.unique(dates_trainval)
    blocks = []
    for i in range(len(unique_dates)):
        blocks.append(np.where(dates_trainval == unique_dates[i])[0])

    np.random.seed(1)
    np.random.shuffle(blocks)
    shuffled_indices = np.asarray(list(itertools.chain.from_iterable(blocks)))

    return shuffled_indices

In [ ]:
def cv_split(split_data, fold_index, num_fold):
    '''
    input:
    split_data: the dayblock shuffled indices to be splitted
    fold_index: the ith fold chosen as the validation, used for generating the seed for random shuffling
    num_fold: N-fold cross validation
    output:
    data_train: the train data indices
    data_val: the validation data indices
    '''
    num_samples = len(split_data)
    indices = np.arange(num_samples)

    val_mask = np.zeros(len(indices), dtype=bool)
    val_mask[int(fold_index / num_fold * num_samples):int((fold_index + 1) / num_fold * num_samples)] = True
    val_indices = indices[val_mask]
    train_indices = indices[np.logical_not(val_mask)]

    np.random.seed(fold_index)
    np.random.shuffle(train_indices)
    np.random.shuffle(val_indices)
    
    data_train = split_data[train_indices]
    data_val = split_data[val_indices]

    return data_train,data_val

In [ ]:
def process_image(image_data):
    '''
    image data processing: reshaping and normalization
    '''
    image_data = tf.transpose(image_data,perm=[0,2,3,1,4])
    image_data = tf.reshape(image_data, [image_data.shape[0],image_data.shape[1],image_data.shape[2],-1])

    image_data = tf.image.convert_image_dtype(image_data, tf.float32)

    return image_data

In [ ]:
def data_loader(hdf5_data_path,sample_idx,batch_size=256):
    '''
    input:
    hdf5_data_path: path to hdf5 data file
    sample_idx: 
        for training and validation:
            dayblock shuffled indices with cross-validation split into training and validation
            either training or validation indices will be input
        for testing: the indices are not shuffled
    is_trainval: a flag, True for trainig and validation
    output:
    dataset: dataset for training, validation
    '''

    def mapping_func_py(hdf5_data_path,sample_idx):
        '''
        mapping indices to corresponding images and irraidance data in hdf5 (python expression)
        '''
        hdf5_data_path = hdf5_data_path.numpy().decode() 
        sample_idx = sorted(sample_idx.numpy())

        with h5py.File(hdf5_data_path,'r') as f:

            images_log = f['trainval']['images_log'][sample_idx]
            si_pred = f['trainval']['si_pred'][sample_idx]
            cloud_fraction = f['trainval']['cloud_fraction'][sample_idx]
        
        
            images_log = process_image(images_log)
                     
            si_pred = tf.convert_to_tensor(si_pred, dtype=tf.float32) 
            cloud_fraction = tf.convert_to_tensor(cloud_fraction, dtype=tf.float32) 
        
            return images_log,cloud_fraction, si_pred #GHI_rad, si_pred  #

    def mapping_func_tf(hdf5_data_path,sample_idx):
        '''
        a wrapper mapping function to get the nested data structure 
        the output type of tf.py_function cannot be a nested sequence when using a tf.py_function with the tf.data API
        '''
        images_log,cloud_fraction, si_pred = tf.py_function(func=mapping_func_py,
                                                           inp=[hdf5_data_path, sample_idx], 
                                                           Tout=(tf.float32, tf.float32,tf.float32))
        return (images_log, cloud_fraction), si_pred
    
    
    idx_ds = tf.data.Dataset.from_tensor_slices(sample_idx)
    
    idx_ds = idx_ds.shuffle(buffer_size = idx_ds.cardinality().numpy(),seed=0)
    idx_ds = idx_ds.batch(batch_size).repeat().prefetch(tf.data.experimental.AUTOTUNE)
    
    dataset = idx_ds.map(lambda x: mapping_func_tf(hdf5_data_path,x),
                         num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    return dataset

## 3. Model Architecture

In [ ]:
num_filters = 24
kernel_size = [3,3]
pool_size = [2,2]
strides = 2
dense_size = 1024
drop_rate = 0.4

num_epochs = 200 #(The maximum epoches set to 200 and there might be early stopping depends on validation loss)
num_fold = 10 # 10-fold cross-validation
batch_size = 256
learning_rate = 1e-05

In [ ]:
def cnn_lstm_model():
    x_in = keras.Input(shape=image_input_dim)
    
    x2_in = keras.Input(shape=num_log_term)

    x = keras.layers.Conv2D(num_filters,kernel_size,padding="same",activation='relu')(x_in)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size, strides)(x)

    x = keras.layers.Conv2D(num_filters*2,kernel_size,padding="same",activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D(pool_size, strides)(x)
    x = keras.layers.Flatten()(x)
    
    x = keras.layers.Concatenate(axis=1)([x, x2_in])
    
    x = keras.layers.Reshape((1, -1))(x) 
    
    lstm_model =keras.layers.RNN(keras.layers.LSTMCell(units=32))(x)
    cnn_lstm = keras.layers.Dense(dense_size, activation='relu')(lstm_model)
    cnn_lstm = keras.layers.Dropout(drop_rate)(cnn_lstm)
    cnn_lstm = keras.layers.Dense(dense_size, activation='relu')(cnn_lstm)
    cnn_lstm = keras.layers.Dropout(drop_rate)(cnn_lstm)

    y_out = keras.layers.Dense(units=1)(cnn_lstm)

    model = keras.Model(inputs=[x_in, x2_in],outputs=y_out)

    return model
cnn_lstm_model().summary()

## 3. Model Training

In [ ]:
indices_dayblock_shuffled = day_block_shuffle(times_trainval)

train_loss_hist = []
val_loss_hist = []
    
for i in range(num_fold):
    
    keras.backend.clear_session()
    model = cnn_lstm_model()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate),loss='mse')
    
    print('Repetition {0} model training started ...'.format(i+1))
    
    save_directory = os.path.join(output_folder,'repetition_'+str(i+1))
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
        
    indices_train, indices_val = cv_split(indices_dayblock_shuffled,i,num_fold)
    
    ds_train_batched = data_loader(data_path,indices_train)#, batch_size=batch_size)
    ds_val_batched = data_loader(data_path,indices_val,batch_size=600)

    earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(save_directory,'best_model_repitition_'+str(i+1)+'.h5'), 
                                monitor='val_loss', mode='min', save_best_only=True, verbose=1)

    history = model.fit(ds_train_batched, epochs=num_epochs, steps_per_epoch=len(indices_train)//batch_size+1,
                                   verbose=1, callbacks=[earlystop,checkpoint], validation_data=ds_val_batched,
                                  validation_steps=len(indices_val)//batch_size+1)
    train_loss_hist.append(history.history['loss'])
    val_loss_hist.append(history.history['val_loss'])

    plt.plot(train_loss_hist[i],label='train')
    plt.plot(val_loss_hist[i],label='validation')
    plt.legend()
    plt.show()

    np.save(os.path.join(output_folder,'train_loss_hist.npy'),train_loss_hist)
    np.save(os.path.join(output_folder,'val_loss_hist.npy'),val_loss_hist)

In [ ]:
best_train_loss_MSE = np.zeros(num_fold)
best_val_loss_MSE = np.zeros(num_fold)

for i in range(num_fold):
    best_val_loss_MSE[i] = np.min(val_loss_hist[i])
    idx = np.argmin(val_loss_hist[i])
    best_train_loss_MSE[i] = train_loss_hist[i][idx]
    print('Model {0}  -- train loss: {1:.2f}, validation loss: {2:.2f} (RMSE)'.format(i+1, np.sqrt(best_train_loss_MSE[i]), np.sqrt(best_val_loss_MSE[i])))
print('The mean train loss (RMSE) for all models is {0:.2f}'.format(np.mean(np.sqrt(best_train_loss_MSE))))
print('The mean validation loss (RMSE) for all models is {0:.2f}'.format(np.mean(np.sqrt(best_val_loss_MSE))))

## 3. Model Testing

In [ ]:
times_test = np.load(os.path.join(data_folder,'forecast_data',"times_test.npy"),allow_pickle=True)
print("times_test.shape:", times_test.shape)

with h5py.File(data_path,'r') as f:

    images_log_test = f['test']['images_log'][...]
    si_pred_test = f['test']['si_pred'][...]
    si_log_test = f['test']['si_log'][...]
    cloud_fraction_test = f['test']['cloud_fraction'][...]
    
    
images_log_test = np.transpose(images_log_test,axes=[0,2,3,1,4])
images_log_test = np.reshape(images_log_test, [images_log_test.shape[0],images_log_test.shape[1],images_log_test.shape[2],-1])
images_log_test = (images_log_test/255.0).astype('float32')


    
print("images_log_test.shape:",images_log_test.shape)
print("si_pred_test.shape:",si_pred_test.shape)
print("si_log_test.shape:",si_log_test.shape)
print("cloud_fraction_test.shape:",cloud_fraction_test.shape)

In [ ]:
loss = np.zeros((num_fold,len(times_test)))
prediction = np.zeros((num_fold,len(times_test)))

with tf.device ('CPU'):
    
    for i in range(num_fold):
        print("loading repetition {0} model ...".format(i+1))
        model_path = os.path.join(output_folder,'repetition_'+str(i+1),'best_model_repitition_'+str(i+1)+'.h5')
        model = keras.models.load_model(model_path)#,  compile=False)
        
        
        
        print("evaluating performance for the model".format(i+1))
        loss[i] = model.evaluate(x=[images_log_test,cloud_fraction_test], y=si_pred_test, batch_size=200, verbose=1)
        
        print("generating predictions for the model".format(i+1))
        prediction[i] = np.squeeze(model.predict([images_log_test,cloud_fraction_test], batch_size=200, verbose=1))
        prediction[i] =  prediction[i]*1000
        
si_pred_test = si_pred_test*1000

np.save(os.path.join(output_folder,'test_predictions.npy'),prediction)

print('-'*50)
print("model ensembling ...")
prediction_ensemble = np.mean(prediction,axis=0)
loss_ensemble = np.sqrt(np.mean((prediction_ensemble-si_pred_test)**2))
print("the test set RMSE is {0:.3f} for the ensemble model".format(loss_ensemble))

In [ ]:

sunny_dates =[(2024,7,29)]
partially_cloudy_dates = [(2024,7,28)]
cloudy_dates =[(2024,7,27)]


sunny_dates_test = [datetime.date(day[0],day[1],day[2]) for day in sunny_dates]
partially_cloudy_dates_test = [datetime.date(day[0],day[1],day[2]) for day in partially_cloudy_dates]
cloudy_dates_test = [datetime.date(day[0],day[1],day[2]) for day in cloudy_dates]

dates_test = np.asarray([times.date() for times in times_test])

mask_sunny_dates = np.zeros(len(si_pred_test),dtype=bool)
for i in sunny_dates_test:
    mask_sunny_dates[np.where(dates_test==i)[0]]=1

mask_partially_cloudy_dates = np.zeros(len(si_pred_test),dtype=bool)
for i in partially_cloudy_dates_test:
    mask_partially_cloudy_dates[np.where(dates_test==i)[0]]=1

mask_cloudy_dates =np.zeros(len(si_pred_test),dtype=bool)
for i in cloudy_dates_test:
    mask_cloudy_dates[np.where(dates_test==i)[0]]=1

    
times_test_sunny = times_test[mask_sunny_dates]
si_pred_test_sunny = si_pred_test[mask_sunny_dates]
si_log_test_sunny = si_log_test[mask_sunny_dates]
prediction_ensemble_sunny = prediction_ensemble[mask_sunny_dates]
print("times_test_sunny.shape:",times_test_sunny.shape)


times_test_partially_cloudy = times_test[mask_partially_cloudy_dates]
si_pred_test_partially_cloudy = si_pred_test[mask_partially_cloudy_dates]
si_log_test_partially_cloudy = si_log_test[mask_partially_cloudy_dates]
prediction_ensemble_partially_cloudy = prediction_ensemble[mask_partially_cloudy_dates]
print("times_test_partially_cloudy.shape:",times_test_partially_cloudy.shape)

times_test_cloudy = times_test[mask_cloudy_dates]
si_pred_test_cloudy= si_pred_test[mask_cloudy_dates]
si_log_test_cloudy = si_log_test[mask_cloudy_dates]
prediction_ensemble_cloudy = prediction_ensemble[mask_cloudy_dates]
print("times_test_cloudy.shape:",times_test_cloudy.shape)

In [ ]:
import pandas as pd
df = pd.DataFrame(si_pred_test)

df1=df[:863].squeeze()
df1 =df1.shift(15)
df1=df1.replace([np.nan, -np.inf], 0)

df2=df[863:865+861]#.squeeze()
df2 =df2.shift(15)
df2=df2.replace([np.nan, -np.inf], 0)


df3=df[865+861:]#.squeeze()
df3 =df3.shift(15)
df3=df3.replace([np.nan, -np.inf], 0)

cnn_lstm_prediction =pd.DataFrame(prediction_ensemble.squeeze())
testing_timeas=pd.DataFrame(times_test.squeeze())




df.to_csv('pesist.csv')

In [ ]:
df.to_csv('pesist.csv') 
cnn_lstm_prediction.to_csv('15min_cnn_lstm.csv') 
testing_timeas.to_csv("test_times.csv")
df1.to_csv('clear_persist.csv')
df2.to_csv('partly_persist.csv')
df3.to_csv('cloudy_persist.csv')

In [ ]:
persistence_prediction = pd.concat([df1,df2,df3]).squeeze().to_numpy() 

sunny_persistence_predictions = df1.to_numpy()

partly_cloudy_persistence_predictions = (df2.to_numpy())

cloudy_persistence_predictions = df3.to_numpy()


rmse_sunny_persistence = np.sqrt(np.mean(np.square((sunny_persistence_predictions-si_pred_test_sunny))))
rmse_partly_cloudy_persistence = np.sqrt(np.mean(np.square((partly_cloudy_persistence_predictions-si_pred_test_partially_cloudy))))
rmse_cloudy_persistence = np.sqrt(np.mean(np.square((cloudy_persistence_predictions-si_pred_test_cloudy))))
rmse_persistence_overall = np.sqrt((rmse_sunny_persistence**2*len(si_pred_test_sunny)+rmse_cloudy_persistence **2*len(si_pred_test_cloudy)+rmse_partly_cloudy_persistence**2*len(si_pred_test_partially_cloudy))/(len(si_pred_test)))

print("sunny days persistence RMSE: {0:.3f}".format(rmse_sunny_persistence))
print("partly cloudy days persistence RMSE: {0:.3f}".format(rmse_partly_cloudy_persistence))
print("cloudy days persistence RMSE: {0:.3f}".format(rmse_cloudy_persistence))
print("persistence overall RSME: {0:.2f}".format(rmse_persistence_overall))

In [ ]:
mae_persistnce_sunny = np.mean(np.abs((sunny_persistence_predictions-si_pred_test_sunny)))
mae_persistnece_partially_cloudy = np.mean(np.abs((partly_cloudy_persistence_predictions-si_pred_test_partially_cloudy)))
mae_persistence_cloudy = np.mean(np.abs((cloudy_persistence_predictions-si_pred_test_cloudy)))
mae_persistence_overall = (mae_persistence_cloudy*len(si_pred_test_cloudy) + mae_persistnece_partially_cloudy*len(si_pred_test_partially_cloudy)+mae_persistnce_sunny*len(si_pred_test_sunny))/(len(si_pred_test))

print("persistence sunny days MAE: {0:.2f}".format(mae_persistnce_sunny))
print("persistence partially cloudy days MAE: {0:.2f}".format(mae_persistnece_partially_cloudy))
print("persistence cloudy days MAE: {0:.2f}".format(mae_persistence_cloudy))
print("persistennce overall MAE: {0:.2f}".format(mae_persistence_overall))

In [ ]:
rmse_sunny = np.sqrt(np.mean(np.square((prediction_ensemble_sunny-si_pred_test_sunny))))
rmse_partially_cloudy=np.sqrt(np.mean(np.square((prediction_ensemble_partially_cloudy-si_pred_test_partially_cloudy))))
rmse_cloudy = np.sqrt(np.mean(np.square((prediction_ensemble_cloudy-si_pred_test_cloudy))))
rmse_overall = np.sqrt((rmse_sunny**2*len(si_pred_test_sunny)+rmse_cloudy**2*len(si_pred_test_cloudy)+rmse_partially_cloudy**2*len(si_pred_test_partially_cloudy))/(len(si_pred_test)))

print("test set sunny days RMSE: {0:.3f}".format(rmse_sunny))
print("test set partially cloudy days RMSE: {0:.3f}".format(rmse_partially_cloudy))
print("test set cloudy days RMSE: {0:.3f}".format(rmse_cloudy))
print("test set overall RMSE: {0:.3f}".format(rmse_overall))

In [ ]:
mae_sunny = np.mean(np.abs((prediction_ensemble_sunny-si_pred_test_sunny)))
mae_partially_cloudy = np.mean(np.abs((prediction_ensemble_partially_cloudy-si_pred_test_partially_cloudy)))
mae_cloudy = np.mean(np.abs((prediction_ensemble_cloudy-si_pred_test_cloudy)))
mae_overall = (mae_cloudy*len(si_pred_test_cloudy) + mae_partially_cloudy*len(si_pred_test_partially_cloudy)+mae_sunny*len(si_pred_test_sunny))/(len(si_pred_test))

print("test set sunny days MAE: {0:.2f}".format(mae_sunny))
print("test set partially cloudy days MAE: {0:.2f}".format(mae_partially_cloudy))
print("test set cloudy days MAE: {0:.2f}".format(mae_cloudy))
print("test set overall MAE: {0:.2f}".format(mae_overall))

In [ ]:
dates_test = np.array([dtinfo.date() for dtinfo in times_test])
hours_test = np.array([dtinfo.time() for dtinfo in times_test])

f,axarr = plt.subplots(2,3,sharex=True, sharey = True)
xfmt = mdates.DateFormatter('%H:%M')
fmt_date = datetime.date(2000,1,1)

green = '#8AB8A7'
red = '#B83A4B'
blue = '#67AFD2'
grey =  '#B6B1A9'

red1= "#FA3C3C"


for i,date in enumerate(sunny_dates_test):
    ax = axarr[i,0]
    date_mask = (dates_test == date)
    
    hours_xaxis= [datetime.datetime.combine(fmt_date, hour) for hour in hours_test[date_mask]] 
  
    rmse = np.sqrt(np.mean(np.square((si_pred_test[date_mask]-prediction_ensemble[date_mask]))))
    rmse_persistence=np.sqrt(np.mean(np.square((si_pred_test[date_mask]-persistence_prediction[date_mask])))) 

    mae = np.mean(np.abs((si_pred_test[date_mask]-prediction_ensemble[date_mask])))
    mae_persistence = np.mean(np.abs((si_pred_test[date_mask]-persistence_prediction[date_mask])))
    
    
    ax.plot(hours_xaxis, si_pred_test[date_mask], linewidth = 2,color= "k", label = 'Measured')
   
    ax.plot(hours_xaxis, prediction_ensemble[date_mask],linewidth = 2,label = 'CNN-LSTM',color= 'tab:red',markerfacecolor="None")
    ax.plot(hours_xaxis,persistence_prediction[date_mask] ,linewidth = 2,label = ' Persistence',color="tab:blue",markerfacecolor="None") 
    
    ax.set_ylabel('GHI (W/m2)')
    ax.xaxis.set_major_formatter(xfmt)
    ax.text(0.05,0.88,"RMSE: {0:.2f}\nMAE: {1:.2f}".format(rmse,mae),transform=ax.transAxes,color='tab:red', fontsize="12")


for i,date in enumerate(partially_cloudy_dates_test):
    ax = axarr[i,1]
    date_mask = (dates_test == date)
    hours_xaxis= [datetime.datetime.combine(fmt_date, hour) for hour in hours_test[date_mask]] 
    
    rmse = np.sqrt(np.mean(np.square((si_pred_test[date_mask]-prediction_ensemble[date_mask]))))

    mae = np.mean(np.abs((si_pred_test[date_mask]-prediction_ensemble[date_mask])))
    
    ax.plot(hours_xaxis, si_pred_test[date_mask], linewidth = 2,color='k')
    ax.plot(hours_xaxis, prediction_ensemble[date_mask],linewidth =2,label = 'CNN-LSTM',color="tab:red",markerfacecolor="None")

    ax.xaxis.set_major_formatter(xfmt)
    ax.text(0.05,0.88,"RMSE: {0:.2f}\nMAE: {1:.2f}".format(rmse,mae),transform=ax.transAxes,color='tab:red', fontsize="12")
    
    
for i,date in enumerate(cloudy_dates_test):
    ax = axarr[i,2]
    date_mask = (dates_test == date)
    hours_xaxis= [datetime.datetime.combine(fmt_date, hour) for hour in hours_test[date_mask]] 
    
    rmse = np.sqrt(np.mean(np.square((si_pred_test[date_mask]-prediction_ensemble[date_mask]))))

    mae = np.mean(np.abs((si_pred_test[date_mask]-prediction_ensemble[date_mask])))
    
    ax.plot(hours_xaxis, si_pred_test[date_mask], linewidth =2,color='k')
    ax.plot(hours_xaxis, prediction_ensemble[date_mask],linewidth =2,label = 'CNN-LSTM',color='tab:red',markerfacecolor="None")


    
    ax.xaxis.set_major_formatter(xfmt)
    ax.text(0.05,0.88,"RMSE: {0:.2f}\nMAE: {1:.2f}".format(rmse,mae),transform=ax.transAxes,color='tab:red', fontsize="12")
    
axarr[0,0].set_ylim(0, 1200)
axarr[0,0].legend(bbox_to_anchor= [1.6,1.3], loc = 'upper center', ncol = 3)
axarr[-1,0].set_xlabel('Time')
axarr[-1,1].set_xlabel('Time')
axarr[-1,-1].set_xlabel('Time')

axarr[0,0].legend(loc='upper right', fontsize="12") 

f.set_size_inches(18,12)
f.tight_layout(pad=1.5) 

plt.show()